# 虚拟内存

毋庸置疑，虚拟内存绝对是操作系统中最重要的概念之一。我想主要是由于内存的重要性地位。CPU太快但容量小且功能单一，其他I/O硬件支持各种花式功能但是现对于CPU又太慢。于是虚拟内存作为一种润滑剂就开始大显身手。

而现在操作系统中，多任务已是标配。多任务并行，大大提升了CPU的利用率，但是却引出了多个进程对内存操作的冲突问题，虚拟内存就是未来解决这个问题而提出的。

**虚拟内存**：是计算机系统内存管理的一种技术。它使得**应用程序**认为它拥有连续的可用的**内存**，实际上，它通常是被分隔成多个**物理内存碎片**，还有部分暂时存储在外部*磁盘存储器*上，在需要时进行数据交换。

在windows系统中：虚拟内存是Windows 为作为内存使用的一部分硬盘空间。虚拟内存在硬盘上其实就是为一个硕大无比的文件，文件名是PageFile.Sys，通常状态下是看不到的。必须关闭资源管理器对系统文件的保护功能才能看到这个文件。虚拟内存有时候也被称为是“页面文件”就是从这个文件的文件名中来的。

## 分页和页表

虚拟内存是操作系统里的概念，对操作系统来说，虚拟内存就是一张张的对照表，即获取虚拟内存里的数据时应该去物理内存对应的地址找。

我们知道系统的基本单位都是Byte字节，如果将每一个虚拟内存的Byte字节都对应到物理内存的地址，每个条目最少需要8字节，在4G内存的情况下，就需要32G的空间来存放对照表，那么这张表就大的真正的物理内存也放不下了，于是操作系统以引入了页（Page）的概念。

在系统启动时，操作系统将整个物理内存以4K为单位，划分为各个页。之后进行内存分配时都以页为单位，那么虚拟内存对应的物理内存页的映射表就大大的减小了，4G内存只需要8M的映射表即可。操作系统虚拟内存到物理内存的映射表，就被称为**页表**。


## 内存寻址和分配

我们知道通过虚拟内存机制，每个进程都以为自己占用了全部内存，进程访问内存时，操作系统都会把进程提供的虚拟内存地址转换为物理地址，再去对应的物理地址上获取数据。CPU中有一种硬件，内存管理单元MMU（memory management unit）专门用来翻译虚拟内存地址。


# 功能

虚拟内存不仅通过内存地址转换解决了多个进程访问内存冲突的问题，还带来更多的益处。

### 进程内存管理

它有助有进程进行内存管理，主要体现在：

+ 内存完整性：我们在编写应用程序的时候不用考虑大块地址的分配，总是认为系统有足够的内存即可。
+ 安全：由于进程访问内存时，都要通过页表来寻址，操作系统在页表的各个项目上添加各种访问权限标识位，就可以实现内存的权限控制。


### 数据共享

通过虚拟内存更容易实现内存和数据的共享。

在进程加载系统库时，总是先分配一块内存，将磁盘中的库文件加载到这块内存中，在直接使用物理内存时，由于物理内存地址唯一，即系统发现同一个库在系统内加载了两次，但每个进程指定的加载内存不一样，系统也无能为力。

而在使用虚拟内存时，系统只需要将进程的虚拟内存地址指向库文件所在的物理内存地址即可。

### Swap

**Swap分区**，即**交换区**在系统的物理内存不够用的时候，把物理内存中的一部分空间释放出来，以供当前运行的程序使用。那些被释放的空间可能来自与一些很长时间没有什么操作的程序，这些被释放的空间被临时保存到Swap分区中，等到这些程序要运行的时候再从Swap分区中恢复保存保存的数据到内存中。


#  常见问题

### 32位和64位

CPU通过物理总线访问内存，那么访问地址的范围就受限于机器总线的数量，在32位机器上有32条总线，每条总线有高低两种电位分别代表bit的0和1，那么可以访问的最大地址就是2^32bit=4GB，所以32位机器插入大于4G的内存是无效的，CPU访问不到多于4G的内存。

但64位机器并没有64位总线，而且其最大内存还要受限与操作系统，linux目前支持最大256G内存。

根据虚拟内存的概念，在32位系统上运行64位软件并无不可，但由于系统对于虚拟内存地址结构设计，64位的虚拟地址在32位系统内并不能使用。

### 直接操作物理内存

操作系统使用了虚拟内存，我们想要直接操作内存该怎么办呢？

linux会将各个设备都映射到 /dev/ 目录下的文件，我们可以通过这些设备文件直接操作硬件，内存也不例外。在linux中，内存设置被映射为 /dev/mem ，root用户通过对这个文件读写可以直接操作内存。


### Swap 操作

虚拟内存的Swap特性并不是总有益的，放任进程不停的将数据在内存与磁盘之间大量交换会极大地占用CPU，降低系统运行效率，所以有时候我们并不希望使用Swap。

我们可以修改 vim.swappiness=o 来设置内存尽量少使用 Swap，或者干脆使用 swapoff 命令禁用掉 Swap。
